In [19]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

#import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library


In [18]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [20]:
import folium # map rendering library

#### scrap the wikipedia page read the Post Code table and convert into dataframe

In [3]:
df_canada = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0,  index_col=None, skiprows=None)
df = pd.DataFrame(df_canada[0])

#### Ignore cells with a borough that is Not assigned

In [4]:
df_dropna= df[df.Borough != 'Not assigned']

#### Combine neighborhood in one postal code area, separte values by comma

In [5]:
df_concat = df_dropna.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join }).reset_index()


#### Assign neighbourhood as brough when neighbourhood has value "Not assigned"

In [6]:

df_concat.loc[df_concat.Neighbourhood== 'Not assigned', 'Neighbourhood'] = df_concat.Borough

In [7]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')


#### Read the Geocoder csv file to get geographical coordinates information

In [8]:
df_concat = df_concat.rename(columns={'Postcode': 'Postal Code'})


#### join 2 dataframes

In [9]:
mergedDf = df_concat.merge(df_geo)

In [10]:
mergedDf.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
df_toronto= mergedDf[mergedDf['Borough'].str.contains('Toronto')]

In [14]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


In [22]:
#Use geopy library to get the latitude and longitude values of New York City.
#In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [48]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [56]:
CLIENT_ID = '559066510' # your Foursquare ID
CLIENT_SECRET = 'Fs1111!' # your Foursquare Secret
VERSION = '20191005' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 559066510
CLIENT_SECRET:Fs1111!


In [ ]:
#Let's explore the first neighborhood in our dataframe.
#Get the neighborhood's name.

In [35]:
df_toronto.loc[:,'Neighbourhood']

37                                          The Beaches
41                         The Danforth West, Riverdale
42                       The Beaches West, India Bazaar
43                                      Studio District
44                                        Lawrence Park
45                                     Davisville North
46                                   North Toronto West
47                                           Davisville
48                          Moore Park, Summerhill East
49    Deer Park, Forest Hill SE, Rathnelly, South Hi...
50                                             Rosedale
51                          Cabbagetown, St. James Town
52                                 Church and Wellesley
53                            Harbourfront, Regent Park
54                             Ryerson, Garden District
55                                       St. James Town
56                                          Berczy Park
57                                   Central Bay

In [38]:
#et the neighborhood's latitude and longitude values.
neighborhood_latitude = df_toronto.loc[:, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[:, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[:, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of 37                                          The Beaches
41                         The Danforth West, Riverdale
42                       The Beaches West, India Bazaar
43                                      Studio District
44                                        Lawrence Park
45                                     Davisville North
46                                   North Toronto West
47                                           Davisville
48                          Moore Park, Summerhill East
49    Deer Park, Forest Hill SE, Rathnelly, South Hi...
50                                             Rosedale
51                          Cabbagetown, St. James Town
52                                 Church and Wellesley
53                            Harbourfront, Regent Park
54                             Ryerson, Garden District
55                                       St. James Town
56                                          Berczy Park
57             

In [54]:
#Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
#First, let's create the GET request URL. Name your URL url.
# The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [ ]:
#Send the GET request and examine the resutls

In [58]:
results = requests.get(url).json()
results

In [ ]:
#From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.


In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#Now we are ready to clean the json and structure it into a pandas dataframe.

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))